In [29]:
from common_functions import google_sheets, task_fail_slack_alert, get_secret, snowflake_query, ret_metabase
from datetime import datetime, timedelta
import datetime as dt

import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
import sqlalchemy
import psycopg2
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

def initialize_env():
    snowflake_sg_secret = json.loads(get_secret("Snowflake-sagemaker"))
    slack_secret = json.loads(get_secret("prod/slack/reports"))
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    dwh_writer_secret = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))

    os.environ["SNOWFLAKE_USERNAME"] = snowflake_sg_secret["username"]
    os.environ["SNOWFLAKE_PASSWORD"] = snowflake_sg_secret["password"]
    os.environ["SNOWFLAKE_ACCOUNT"] = snowflake_sg_secret["account"]
    os.environ["SNOWFLAKE_DATABASE"] = snowflake_sg_secret["database"]

    os.environ["SLACK_TOKEN"] = slack_secret["token"]

    os.environ["FINTECH_EMONEY_EMAIL"] = fintech_service_account["email_name"]
    os.environ["FINTECH_EMONEY_PASSWORD"] = fintech_service_account["email_password"]

    metabase_secret = json.loads(get_secret("prod/metabase/maxab_config"))
    os.environ["EGYPT_METABASE_USERNAME"] = metabase_secret["metabase_user"]
    os.environ["EGYPT_METABASE_PASSWORD"] = metabase_secret["metabase_password"]

    os.environ["DWH_WRITER_HOST_NEW"] = dwh_writer_secret["host"]
    os.environ["DWH_WRITER_NAME_NEW"] = dwh_writer_secret["dbname"]
    os.environ["DWH_WRITER_USER_NAME_NEW"] = dwh_writer_secret["username"]
    os.environ["DWH_WRITER_PASSWORD_NEW"] = dwh_writer_secret["password"] 

    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets

def ret_metabase(country, question, filters={}, initialized = False):

    
    if not initialized: 
        initialize_env()
    
    question_id = str(question)
    
    if country.lower() == 'egypt':
        base_url = 'https://bi.maxab.info/api'
        username = str(os.environ["EGYPT_METABASE_USERNAME"])
        password = str(os.environ["EGYPT_METABASE_PASSWORD"])
    else:
        base_url = 'https://bi.maxabma.com/api'
        username = str(os.environ["AFRICA_METABASE_USERNAME"])
        password = str(os.environ["AFRICA_METABASE_PASSWORD"])

    base_headers = {'Content-Type': 'application/json'}

    try:
        s_response = requests.post(
            base_url + '/session',
            data=json.dumps({
                'username': username,
                'password': password
            }),
            headers=base_headers)
        
        s_response.raise_for_status()

        session_token = s_response.json()['id']
        base_headers['X-Metabase-Session'] = session_token
        
        params = []
        
        for name, value in filters.items():
            filter_type, filter_value = value
            param = {'target': ['variable', ['template-tag', name]], 'value': filter_value}
            
            if filter_type.lower() == 'date':
                param['type'] = 'date/range' if isinstance(filter_value, list) else 'date/single'
            elif filter_type.lower() == 'category':
                param['type'] = 'category'
            elif filter_type.lower() == 'text':
                param['type'] = 'text'
            elif filter_type.lower() == 'number':
                param['type'] = 'number'
            elif filter_type.lower() == 'field list':
                param['type'] = 'id'
                param['target'] = ['dimension', ['template-tag', name]]
            
            params.append(param)

        p_response = requests.post(base_url + '/card/' + question_id + '/query/csv', 
                                   json={'parameters': params}, 
                                   headers=base_headers)
        p_response.raise_for_status()

        my_dict = p_response.content
        s = str(my_dict, 'utf-8')
        my_dict = StringIO(s)
        df = pd.read_csv(my_dict)
        return(df)
    
    except Exception as e:
        logger.error(f"An error occurred: {e}", exc_info=True)
        raise



def check_distribution(df, agent_list):
    """
    Checks if the distribution of project types across agents is even.
    Returns a dictionary with the count of rows per agent for each project type.
    """
    distribution = {}
    project_types = df['project_name'].unique()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        distribution[project] = project_df['agent_assigned'].value_counts().reindex(agent_list, fill_value=0)
    
    return distribution

def redistribute_rows(df, agent_list):
    """
    Redistributes rows among agents if the distribution is uneven.
    """
    project_types = df['project_name'].unique()
    redistributed_data = pd.DataFrame()
    print("Redistributing rows...")
    for project in project_types:
        project_df = df[df['project_name'] == project]
        project_df = project_df.sample(frac=1).reset_index(drop=True)  # Shuffle data
        rows_per_agent = len(project_df) // len(agent_list)
        remainder = len(project_df) % len(agent_list)
        
        start_idx = 0
        
        for i, agent in enumerate(agent_list):
            end_idx = start_idx + rows_per_agent + (1 if i < remainder else 0)
            agent_data = project_df.iloc[start_idx:end_idx].copy()
            agent_data['agent_assigned'] = agent
            
            redistributed_data = pd.concat([redistributed_data, agent_data])
            
            start_idx = end_idx
    
    redistributed_data = redistributed_data.reset_index(drop=True)
    
    return redistributed_data

def ensure_correct_dispatching(df, agent_list, final_old_assign):
    """
    Ensures the dispatching is done correctly by checking and redistributing rows if necessary.
    `final_old_assign` rows are excluded from redistribution.
    """

    # Exclude final_old_assign from the distribution check, but keep rows with main_system_id == 1
    new_assignments = df[~df['main_system_id'].isin(final_old_assign['main_system_id']) | (df['main_system_id'] == 1)]
    
    distribution = check_distribution(new_assignments, agent_list)
    
    # Check if any project has an uneven distribution across agents
    uneven_distribution = any(distribution[project].nunique() > 1 for project in distribution)
    
    if uneven_distribution:
        print("Uneven distribution detected.")
        new_assignments = redistribute_rows(new_assignments, agent_list)
    else:
        print("Distribution is even. No redistribution needed.")
    
    # Combine the redistributed new assignments with the final_old_assign
    final_data = pd.concat([final_old_assign, new_assignments], ignore_index=True)
    
    return final_data

def check_distribution_df(df, agent_list):
    """
    Checks if the distribution of project types across agents is even.
    Returns a DataFrame with the count of rows per agent for each project type.
    """
    project_types = df['project_name'].unique()
    
    distribution_data = []
    current_time = datetime.now()  # Get the current datetime
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        agent_counts = project_df['agent_assigned'].value_counts().reindex(agent_list, fill_value=0)
        
        for agent, count in agent_counts.items():
            distribution_data.append({
                'project_type': project,
                'agent_assigned': agent,
                'count': count,
                'datetime': dt.datetime.now()  + timedelta(hours=3)
            })
    
    distribution_df = pd.DataFrame(distribution_data)
    
    return distribution_df

def clean_column_id(df, column_name):
    # Ensure the column is treated as a string
    df[column_name] = df[column_name].astype(str)
    
    # Replace commas in the string
    df[column_name] = df[column_name].str.replace(',', '')
    
    # Convert back to an integer, if appropriate
    df[column_name] = df[column_name].astype('Int64', errors='ignore')
    
    return df

# ----------------------------------------
# Random distribution (Equal assigning)   
# ----------------------------------------
def assign_data_equal_projects(df, list):
    df = df.sample(frac=1)  # Shuffle the data
    project_types = df['project_name'].unique()
    
    assigned_data = pd.DataFrame()
    
    for project in project_types:
        project_df = df[df['project_name'] == project]
        project_df = project_df.reset_index(drop=True)
        rows_per_agent = len(project_df) // len(list)
        remainder = len(project_df) % len(list)
        
        # Distribute rows equally
        for i, agent in enumerate(list):
            start_idx = i * rows_per_agent
            end_idx = start_idx + rows_per_agent
            agent_data = project_df.iloc[start_idx:end_idx].copy()
            agent_data['agent_assigned'] = agent
            
            # Handle remainder
            if i < remainder:
                extra_row = project_df.iloc[end_idx:end_idx+1].copy()
                extra_row['agent_assigned'] = agent
                agent_data = pd.concat([agent_data, extra_row])
            
            assigned_data = pd.concat([assigned_data, agent_data])
    
    assigned_data = assigned_data.reset_index(drop=True)
    
    return assigned_data

# ----------------------------------------
# Mapping distribution (Segment-based)
# ----------------------------------------
def assign_data_by_mapping(df, mapping_df):
    # store retail-agent mapping in a dictionary 
    mapping_dict = dict(zip(mapping_df['MAIN_SYSTEM_ID'], mapping_df['AGENT_ID']))
        
    assigned_agents = []

    for retailer_id in df['main_system_id']:
        if retailer_id in mapping_dict:
            assigned_agents.append(mapping_dict[retailer_id])
        else:
            assigned_agents.append(None)
   
    df['agent_assigned'] = assigned_agents
    df = df.reset_index(drop=True)

    return df

def get_available_agents(attendance_df, current_hour):
    """Get list of available task-based agents for the current hour."""
    attendance_copy = attendance_df.copy()
    
    attendance_copy['start_time'] = attendance_copy['start_time'].astype(int)
    attendance_copy['end_time'] = attendance_copy['end_time'].astype(int)
    
    attendance_copy['assignment_start_time'] = attendance_copy['start_time'] - 1
    attendance_copy['assignment_end_time'] = attendance_copy['end_time'] - 1
    
    attendance_copy['assign_data'] = np.where(
        (current_hour >= attendance_copy['assignment_start_time']) & 
        (current_hour <= attendance_copy['assignment_end_time']),
        'yes', 'no')
    
    task_based_agents = attendance_copy.loc[
        (attendance_copy['project'] == 'task_based') & 
        (attendance_copy['assign_data'] == 'yes')]
    
    task_based_list = task_based_agents['agent_id'].values.tolist()
    print(f"Number of available agents: {len(task_based_list)}")
    return task_based_list

def fetch_and_process_queries(query_ids, blacklisted_retailers):
    """Fetch and process data from queries, removing blacklisted retailers."""
    queries_for_random = query_ids['Query_id_random'].dropna().astype(int).tolist()
    queries_for_mapped = query_ids['Query_id_segment_based'].dropna().astype(int).tolist()
    print(f"Fetching data from {len(queries_for_random)+len(queries_for_mapped)} queries...")
    
    # Process random queries
    dataframes_R = [ret_metabase("EGYPT", query, initialized=True) for query in queries_for_random]
    empty_queries = []
    for i, df in enumerate(dataframes_R):
        if df.empty:
            empty_queries.append(queries_for_random[i])
        else:
            print(f"Query {queries_for_random[i]} returned {len(df)} records")
        df.columns = map(str.lower, df.columns)
    if empty_queries:
        print(f"WARNING: Queries {empty_queries} returned empty dataframe!")
    # ----------------------------------------
    # write in google sheet available data
    # ----------------------------------------
    # check for empty queries in Random tasks
    for idx, row in query_ids.iterrows():
        query_id = row['Query_id_random']
        if pd.notna(query_id):
            df = ret_metabase("EGYPT", int(query_id), initialized=True)
            query_ids.at[idx, 'Available_data'] = 'Empty' if df.empty else str(len(df))

    # Step 3: Overwrite the sheet with updated full data (including manually-entered columns like F)
    google_sheets('Query ID Assigning', 'Sheet1', 'overwrite', df=query_ids)
    
    # check for empty queries in Segment-based
    for idx, row in query_ids.iterrows():
        query_id = row['Query_id_segment_based']
        if pd.notna(query_id):
            df = ret_metabase("EGYPT", int(query_id), initialized=True)
            query_ids.at[idx, 'Available_data'] = 'Empty' if df.empty else str(len(df))

    # Step 3: Overwrite the sheet with updated full data (including manually-entered columns like F)
    google_sheets('Query ID Assigning', 'Sheet1', 'overwrite', df=query_ids)
    
    # Process mapped queries
    dataframes_M = [ret_metabase("EGYPT", query, initialized=True) for query in queries_for_mapped]
    for i, df in enumerate(dataframes_M):
        if df.empty:
            print(f"WARNING: Query {queries_for_mapped[i]} returned empty dataframe!")
        else:
            print(f"Query {queries_for_mapped[i]} returned {len(df)} records")
        df.columns = map(str.lower, df.columns)
    
    # Combine and clean dataframes
    df_unfiltered_R = pd.concat(dataframes_R, ignore_index=True)
    df_unfiltered_M = pd.concat(dataframes_M, ignore_index=True)
    print(f"Mapping tasks available: {df_unfiltered_M.shape[0]}")
    print(f"Random tasks available: {df_unfiltered_R.shape[0]}")
    
    # Remove blacklisted retailers
    df_raw_R = df_unfiltered_R[~df_unfiltered_R['main_system_id'].isin(blacklisted_retailers)]
    df_raw_M = df_unfiltered_M[~df_unfiltered_M['main_system_id'].isin(blacklisted_retailers)]
    
    df_raw_R = clean_column_id(df_raw_R, 'main_system_id')
    df_raw_M = clean_column_id(df_raw_M, 'main_system_id')
    print(f"Removed {(len(df_unfiltered_R)+len(df_unfiltered_M)) - (len(df_raw_R)+len(df_raw_M))} blacklisted retailers")
    
    return df_raw_R, df_raw_M

def process_previous_assignments(df_raw_R, previous_calls):
    """Process previous assignments and separate new and old assignments."""
    exclude_same_assigns = clean_column_id(previous_calls, 'main_system_id')
    
    # Get new assignments
    task_based = pd.DataFrame(df_raw_R.loc[~df_raw_R['main_system_id'].isin(exclude_same_assigns['main_system_id'].astype(int).values)])
    
    # Get old assignments
    old_assgns = pd.DataFrame(df_raw_R.loc[df_raw_R['main_system_id'].isin(exclude_same_assigns['main_system_id'].astype(int).values)])
    df_1 = old_assgns.merge(previous_calls, on='main_system_id', how='left')
    final_old_assign = df_1[["main_system_id", "retailer_mobile_number", "retailer_name", "description", "reward", "balance", "offer", "agent_assigned", "project_name"]]
    
    return task_based, final_old_assign

In [5]:
def remove_assign(previously_assigned, df, assigns, print_it = False):
    print("Starting remove_assign function...")

    try:
        if not previously_assigned.empty:
            previously_assigned[0] = previously_assigned[0].fillna('').astype(str).str.replace(" ", "", regex=False)
            previously_assigned = previously_assigned.dropna()
            previously_assigned[0] = previously_assigned[0].astype('float').astype('int')
            df['main_system_id'] = df['main_system_id'].astype('int')
            if 'agent_assigned' in df.columns:
                if df['agent_assigned'].dtype == 'object':
                    df.loc[df['agent_assigned'].notna(), 'agent_assigned'] = df.loc[df['agent_assigned'].notna(), 'agent_assigned'].astype(int)

                    print("Converted agent_assigned column to int.")

            filtered_ids = previously_assigned[0].astype(int).values
            main_data_to_assign = df.loc[~df['main_system_id'].isin(filtered_ids) | (df['main_system_id'] == 1)
            ]
            if print_it:
                print(f"Available tasks for this batch after filtering. Rows remaining: {len(main_data_to_assign)}")
                print(f" Retailers that have no aggents to assign to {main_data_to_assign['agent_assigned'].isna().sum()}")

            main_data_to_assign['main_system_id'] = main_data_to_assign['main_system_id'].astype('int')
            main_data_to_assign = main_data_to_assign.groupby('agent_assigned').head(assigns)

            return main_data_to_assign

        else:
            print("Only one previously assigned entry, skipping filter.")
            df['main_system_id'] = df['main_system_id'].astype('int')
            main_data_to_assign = df.groupby('agent_assigned').head(assigns)
            return main_data_to_assign

    except Exception as e:
        print(f"[ERROR] remove_assign failed: {e}")

In [31]:
def task_based_assignment():
    # ----------------------------------------
    # Initialize and get current time
    # ----------------------------------------
    initialize_env()
    now = datetime.now() + timedelta(hours=3)
    hour = int(str(now.time())[0:2])
    print(f"Starting process at hour: {hour}")
    
    # ----------------------------------------
    # Get available agents
    # ----------------------------------------
    attendance = ret_metabase("EGYPT", 13502, initialized=True)
    print(f"Totale agents: {len(attendance)}")
    task_based_list = get_available_agents(attendance, hour)
    time.sleep(15)
    
    if not task_based_list:
        print("No agents available for current hour!")
        return
        
    # ----------------------------------------
    # Get query IDs and blacklisted retailers
    # ----------------------------------------
    query_ids = google_sheets('Query ID Assigning', 'Sheet1', 'get')
    blacklisted_retailers = query_ids['Blacklisted_retailers'].dropna().astype(int).tolist()
    
    # ----------------------------------------
    # Fetch and process query data
    # ----------------------------------------
    df_raw_R, df_raw_M = fetch_and_process_queries(query_ids, blacklisted_retailers)
    
    # ----------------------------------------
    # Process previous assignments / For follow up calls
    # ----------------------------------------
    previous_calls = ret_metabase("EGYPT", 35299, initialized=True)
    previous_calls = clean_column_id(previous_calls, 'main_system_id')
    task_based, final_old_assign = process_previous_assignments(df_raw_R, previous_calls)
    
    # ----------------------------------------
    # Assign agents to tasks
    # ----------------------------------------
    main_data_list = []
    
    # Get and process mapping data
    mapping_df = ret_metabase('Egypt',59587)
    mapping_df = clean_column_id(mapping_df, 'MAIN_SYSTEM_ID')
    mapping_df = clean_column_id(mapping_df, 'AGENT_ID')
    
    if mapping_df.empty:
        print("WARNING: No mapping data found!")
    else:
        print(f"Found {len(mapping_df)} mapping records")
        mapping_df = mapping_df[mapping_df['AGENT_ID'].isin(task_based_list)]
        mapped_data = assign_data_by_mapping(df_raw_M, mapping_df)
        
        if mapped_data.empty:
            print("WARNING: No data mapped to agents!")
        else:
            notna = mapped_data['agent_assigned'].notna().sum()
            na = mapped_data['agent_assigned'].isna().sum()
            print(f"Successfully mapped {notna} records to agents")
            print(f"Retailers unassigned: {na}")
        main_data_list.append(mapped_data)
    
    # Process tasks by priority
    priority_range = range(1, 16)  # 1 to 15 inclusive
    for priority in priority_range:
        priority_df = task_based[task_based["priority"] == priority].reset_index(drop=True)
        main_data = assign_data_equal_projects(priority_df, task_based_list)
        main_data_list.append(main_data)
    
    main_data_total = pd.concat(main_data_list, ignore_index=True)
    print(f"Total assignments available (including ones already assigned): {main_data_total.shape[0]}")
    
    # ----------------------------------------
    # Clean and filter data
    # ----------------------------------------
    # Select required columns
    main_data = main_data_total[
        ["main_system_id", "retailer_mobile_number", "retailer_name", "description", "reward", "balance", "offer", "agent_assigned", "project_name"]]
    
    # Handle special case for main_system_id == 1
    main_system_id_1 = main_data[main_data['main_system_id'] == 1]
    other_main_system_ids = main_data[main_data['main_system_id'] != 1]
    other_main_system_ids = other_main_system_ids.drop_duplicates(subset=['main_system_id'])
    main_data = pd.concat([main_system_id_1, other_main_system_ids], ignore_index=True)
    
    # ----------------------------------------
    # Process previous assignments
    # ----------------------------------------
    sheet = google_sheets('[HOURLY] TASK-BASED Data', 'Data', 'get', [6])
    already_assigned = [[str(val)] for val in sheet.iloc[1:, 0].dropna()]
    already_assigned_df = pd.DataFrame.from_dict(already_assigned)
    
    df = main_data.copy()
    
    # Remove previous assignments and limit retailers per agent
    
    main_data_to_assign = remove_assign(already_assigned_df, df, 40)
    final_old_assign_new = remove_assign(already_assigned_df, final_old_assign, 5)
    
    # Filter for available agents
    filtered_df = main_data_to_assign[main_data_to_assign['agent_assigned'].isin(task_based_list)]
    filtered_df_old = final_old_assign_new[final_old_assign_new['agent_assigned'].isin(task_based_list)]
    
    if filtered_df.empty:
        print("WARNING: No assignments after filtering!")
    else:
        print(f"Number of filtered assignments: {len(filtered_df)}")
    
    # ----------------------------------------
    # Prepare final data
    # ----------------------------------------
    final_data_to_assign = filtered_df.drop_duplicates(subset='main_system_id', keep='first').copy()
    final_old_assign_new = filtered_df_old.drop_duplicates(subset='main_system_id', keep='first').copy()
    
    final_data_to_assign['added_at'] = now
    final_old_assign_new['added_at'] = now
    
    final_data_to_assign = final_data_to_assign.drop(columns='index', errors='ignore')
    final_data_to_assign = ensure_correct_dispatching(final_data_to_assign, task_based_list, final_old_assign_new)
    
    # ----------------------------------------
    # Save data to various destinations
    # ----------------------------------------
    print("Starting data export process...")
    
    # Save to parquet
    final_data_to_assign = final_data_to_assign.astype(str)
    sheet_df = final_data_to_assign[["main_system_id", "retailer_mobile_number", "retailer_name", "description", "reward", "balance", "offer", "agent_assigned", "added_at"]]
    sheet_df
task_based_assignment()

Starting process at hour: 15
Totale agents: 4
Number of available agents: 4
/home/ec2-user/service_account_key.json
Fetching data from 18 queries...
Query 35981 returned 63 records
Query 38188 returned 61 records
Query 59874 returned 62 records
/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
Query 59585 returned 2053 records
Mapping tasks available: 2053
Random tasks available: 186
Removed 1 blacklisted retailers


/tmp/ipykernel_10528/550612972.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].astype(str)
/tmp/ipykernel_10528/550612972.py:224: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.replace(',', '')
/tmp/ipykernel_10528/550612972.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

Found 2053 mapping records
Successfully mapped 1028 records to agents
Retailers unassigned: 1025
Total assignments available (including ones already assigned): 2173
/home/ec2-user/service_account_key.json


NameError: name 'remove_assign' is not defined

In [25]:
# data = ret_metabase('MOROCCO', 10728)

mapping_df = ret_metabase('Egypt',59587)
# mapping_df = google_sheets('EG Telesales Cycle Assignment', 'Sheet1', 'get')
mapping_df = clean_column_id(mapping_df, 'MAIN_SYSTEM_ID')
mapping_df = clean_column_id(mapping_df, 'AGENT_ID')

if mapping_df.empty:
    print("WARNING: No mapping data found!")
else:
    print(f"Found {len(mapping_df)} mapping records")
    # mapping_df = mapping_df[mapping_df['agent_assigned'].isin(task_based_list)]
    # mapped_data = assign_data_by_mapping(df_raw_M, mapping_df)

    # if mapped_data.empty:
    #     print("WARNING: No data mapped to agents!")
    # else:
    #     notna = mapped_data['agent_assigned'].notna().sum()
    #     na = mapped_data['agent_assigned'].isna().sum()
    #     print(f"Successfully mapped {notna} records to agents")
    #     print(f"Retailers unassigned: {na}")
    # main_data_list.append(mapped_data)

Found 2053 mapping records
